### **Imports**

In [245]:
import os

import numpy as np

import random


seed =1380

np.random.seed(seed)

random.seed(seed)

os.environ['PYTHONHASHSEED']=str(seed)

In [246]:
# imports
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
import math
import pylab
from scipy.stats import ttest_ind

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [247]:
adni_with_labels = pd.read_csv('ADNI_genetic_with_labels.csv')
adni_with_labels.shape


KeyboardInterrupt: ignored

### **Genetic data preparation**

In [ ]:
ptid = pd.read_csv('DXSUM_PDXCONV_ADNIALL_03Oct2023.csv')
ptid.head()

In [ ]:
import pandas as pd

# Объедините таблицы по столбцу PTID, оставив только строки, которые есть в обеих таблицах
merged_df = pd.merge(adni_with_labels, ptid, left_on='subjectIDs', right_on='PTID', how='left')

# Удалите ненужные столбцы
columns_to_drop = ['USERDATE2', 'EXAMDATE', 'DXCHANGE', 'DXCURREN', 'DXCONV', 'DXCONTYP', 'DXREV', 'DXNORM', 'DXNODEP', 'DXMCI', 'DXMDES', 'DXMPTR1', 'DXMPTR2', 'DXMPTR3', 'DXMPTR4', 'DXMPTR5', 'DXMPTR6', 'DXMDUE', 'DXMOTHET', 'DXMOTHSP', 'DXDSEV', 'DXDDUE', 'DXAD', 'DXADES', 'DXAPP', 'DXAPROB', 'DXAMETASP', 'DXAOTHRSP', 'DXAPOSS', 'DXAATYSP', 'DXAMETSP', 'DXAOTHSP', 'DXPARK', 'DXPARKSP', 'DXPDES', 'DXPCOG', 'DXPATYP', 'DXPOTHSP', 'DXDEP', 'DXDEPSP', 'DXOTHDEM', 'DXODES', 'DXOOTHSP', 'DXCONFID', 'DIAGNOSIS', 'update_stamp']

merged_df = merged_df.drop(columns=columns_to_drop)

# Теперь в merged_df у вас есть объединенная таблица без ненужных столбцов.


In [ ]:
# Сначала объедините DataFrames по столбцам 'subjectIDs' и 'PTID'
merged_df = adni_with_labels.merge(ptid[['PTID', 'Phase', 'ID', 'RID', 'VISDATE']], left_on='subjectIDs', right_on='PTID', how='left')

# Затем удалите столбец 'PTID', так как он дублирует 'subjectIDs'
merged_df = merged_df.drop('PTID', axis=1)


In [ ]:
# Проверьте, есть ли пустые значения в столбце 'misdate'
if merged_df['VISDATE'].isna().any():
    print("Есть пустые значения в столбце 'VISDATE'")
else:
    print("Нет пустых значений в столбце 'VISDATE'")

    # Подсчитайте количество пустых значений в столбце 'misdate'
count_missing_values = merged_df['VISDATE'].isna().sum()

print(f"Количество пустых значений в столбце 'VISDATE': {count_missing_values}")

In [ ]:
# Проверка на дубликаты в adni_with_labels
duplicates_adni = adni_with_labels[adni_with_labels.duplicated(subset=['subjectIDs'], keep=False)]

# Проверка на дубликаты в ptid
duplicates_ptid = ptid[ptid.duplicated(subset=['PTID'], keep=False)]

# Замените 'merged_df' на имя вашего DataFrame, содержащего столбец "year".
merged_df['year'] = merged_df['year'].astype(str).str.rstrip('.0').astype(int)

# Замените 'merged_df' на имя вашего DataFrame, содержащего столбец "year".
merged_df['year'] = merged_df['year'].astype(str)
merged_df['year'] = merged_df['year'].apply(lambda x: x + '0' if len(x) == 3 else x)
merged_df['year'] = merged_df['year'].astype(int)

# Если тип данных столбца VISDATE не является строковым, преобразуйте его
merged_df['VISDATE'] = merged_df['VISDATE'].astype(str)

# Теперь можно использовать .str accessor
merged_df['VISDATE'] = merged_df['VISDATE'].str[:4]

# Преобразуйте столбцы year и VISDATE в строки, чтобы сравнивать их
merged_df['year'] = merged_df['year'].astype(str)
merged_df['VISDATE'] = merged_df['VISDATE'].astype(str)

# Создайте условие для сравнения year и VISDATE
condition = merged_df['year'] == merged_df['VISDATE']

# Удалите строки, где условие не выполняется
merged_df = merged_df[condition]

unique_count = merged_df['subjectIDs'].nunique()

if unique_count == len(merged_df['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

In [ ]:

# Найдите дублирующиеся значения в столбце 'subjectIDs'
duplicates = merged_df[merged_df.duplicated(subset='subjectIDs', keep=False)]

# Выведите дублирующиеся значения
duplicates

# Определите столбцы, по которым вы хотите проверить дубликаты
columns_to_check = merged_df.columns.difference(['ID'])

# Используйте метод duplicated() для поиска дубликатов по выбранным столбцам
duplicates = merged_df.duplicated(subset=columns_to_check, keep='first')

# Удалите дубликаты, оставив только первое вхождение
filtered_df = merged_df[~duplicates]

genes = filtered_df

In [ ]:
genes.shape

In [ ]:
genes.head()

### **ML for AD vs CN genes**

In [277]:
ad_vs_cn = genes[(genes['label'] == 3.0) | (genes['label'] == 1.0)]

In [278]:
ad = genes[(genes['label'] == 3.0)]
ad.shape

(98, 49393)

In [279]:
mci = genes[(genes['label'] == 2.0)]
mci.shape

(351, 49393)

In [280]:
cn = genes[(genes['label'] == 1.0)]
cn.shape

(245, 49393)

In [281]:
ad_vs_cn.shape

(343, 49393)

In [282]:
ad_vs_cn.head()

,11715100_at,11715101_s_at,11715102_x_at,11715103_x_at,11715104_s_at,11715105_at,11715106_x_at,11715107_s_at,11715108_x_at,11715109_at,11715110_at,11715111_s_at,11715112_at,11715113_x_at,11715114_x_at,11715115_s_at,11715116_s_at,11715117_x_at,11715118_s_at,11715119_s_at,11715120_s_at,11715121_s_at,11715122_at,11715123_s_at,11715124_s_at,11715125_at,11715126_s_at,11715127_s_at,11715128_s_at,11715129_s_at,11715130_s_at,11715131_s_at,11715132_x_at,11715133_s_at,11715134_s_at,11715135_s_at,11715136_at,11715137_s_at,11715138_s_at,11715139_x_at,11715140_s_at,11715141_s_at,11715142_s_at,11715143_s_at,11715144_s_at,11715145_s_at,11715146_x_at,11715147_at,11715148_s_at,11715149_x_at,11715150_s_at,11715151_x_at,11715152_at,11715153_s_at,11715154_s_at,11715155_s_at,11715156_s_at,11715157_s_at,11715158_s_at,11715159_at,11715160_x_at,11715161_s_at,11715162_s_at,11715163_s_at,11715164_s_at,11715165_at,11715166_at,11715167_at,11715168_s_at,11715169_at,11715170_x_at,11715171_at,11715172_at,11715173_s_at,11715174_s_at,11715175_x_at,11715176_at,11715177_s_at,11715178_s_at,11715179_s_at,11715180_s_at,11715181_x_at,11715182_x_at,11715183_s_at,11715184_x_at,11715185_x_at,11715186_x_at,11715187_x_at,11715188_at,11715189_s_at,11715190_s_at,11715191_s_at,11715192_s_at,11715193_s_at,11715194_s_at,11715195_at,11715196_s_at,11715197_s_at,11715198_s_at,11715199_s_at,11715200_s_at,11715201_s_at,11715202_s_at,11715203_s_at,11715204_x_at,11715205_x_at,11715206_x_at,11715207_at,11715208_s_at,11715209_s_at,11715210_s_at,11715211_s_at,11715212_x_at,11715213_s_at,11715214_at,11715215_s_at,11715216_s_at,11715217_s_at,11715218_s_at,11715219_s_at,11715220_s_at,11715221_x_at,11715222_at,11715223_s_at,11715224_s_at,11715225_s_at,11715226_s_at,11715227_s_at,11715228_s_at,11715229_s_at,11715230_s_at,11715231_x_at,11715232_x_at,11715233_s_at,11715234_s_at,11715235_x_at,11715236_s_at,11715237_s_at,11715238_x_at,11715239_x_at,11715240_at,11715241_s_at,11715242_s_at,11715243_x_at,11715244_x_at,11715245_s_at,11715246_x_at,11715247_s_at,11715248_s_at,11715249_s_at,11715250_s_at,11715251_at,11715252_at,11715253_s_at,11715254_s_at,11715255_s_at,11715256_at,11715257_x_at,11715258_s_at,11715259_s_at,11715260_at,11715261_s_at,11715262_at,11715263_at,11715264_s_at,11715265_at,11715266_s_at,11715267_s_at,11715268_s_at,11715269_x_at,11715270_s_at,11715271_s_at,11715272_s_at,11715273_at,11715274_x_at,11715275_s_at,11715276_s_at,11715277_s_at,11715278_s_at,11715279_x_at,11715280_s_at,11715281_at,11715282_s_at,11715283_at,11715284_x_at,11715285_at,11715286_x_at,11715287_at,11715288_s_at,11715289_s_at,11715290_s_at,11715291_s_at,11715292_at,11715293_s_at,11715294_s_at,11715295_s_at,11715296_s_at,11715297_at,11715298_x_at,11715299_s_at,11715300_s_at,11715301_s_at,11715302_s_at,11715303_s_at,11715304_s_at,11715305_s_at,11715306_s_at,11715307_s_at,11715308_s_at,11715309_s_at,11715310_s_at,11715311_s_at,11715312_at,11715313_s_at,11715314_at,11715315_s_at,11715316_x_at,11715317_s_at,11715318_s_at,11715319_s_at,11715320_s_at,11715321_x_at,11715322_s_at,11715323_s_at,11715324_s_at,11715325_at,11715326_s_at,11715327_x_at,11715328_at,11715329_at,11715330_x_at,11715331_x_at,11715332_at,11715333_s_at,11715334_s_at,11715335_at,11715336_x_at,11715337_s_at,11715338_s_at,11715339_at,11715340_s_at,11715341_at,11715342_at,11715343_at,11715344_at,11715345_s_at,11715346_at,11715347_s_at,11715348_x_at,11715349_a_at,...,11764243_at,11764244_s_at,11764245_s_at,11764246_s_at,11764247_s_at,11764248_s_at,11764249_a_at,11764250_at,11764251_s_at,11764252_s_at,11764253_s_at,11764254_a_at,11764255_at,11764256_s_at,11764257_a_at,11764258_at,11764259_at,11764260_s_at,11764261_s_at,11764262_s_at,11764263_s_at,11764264_at,11764265_at,11764266_s_at,11764267_at,11764268_at,11764269_at,11764270_s_at,11764271_at,11764272_at,11764273_at,11764274_s_at,11764275_s_at,11764276_at,11764277_at,11764278_at,11764279_at,11764280_at,11764281_at,11764282_x_at,11764283_s_at,11764284_at,11764285_at,11764286_x_at,11764287_at,11764288_at

In [283]:
#model_to_tune_lr = LogisticRegression()
from sklearn.svm import SVC
#model_to_tune_svm = SVC()
#model_to_tune_rf = RandomForestClassifier()

In [284]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Определите модели для PCA, нормализации и логистической регрессии
pca = PCA()
scaler = StandardScaler()

In [285]:
ad_vs_cn_gene_exp = ad_vs_cn.iloc[:, :-7]

In [286]:
ad_vs_cn_gene_exp.shape

(343, 49386)

In [287]:
ad_vs_cn_gene_exp.head()

,11715100_at,11715101_s_at,11715102_x_at,11715103_x_at,11715104_s_at,11715105_at,11715106_x_at,11715107_s_at,11715108_x_at,11715109_at,11715110_at,11715111_s_at,11715112_at,11715113_x_at,11715114_x_at,11715115_s_at,11715116_s_at,11715117_x_at,11715118_s_at,11715119_s_at,11715120_s_at,11715121_s_at,11715122_at,11715123_s_at,11715124_s_at,11715125_at,11715126_s_at,11715127_s_at,11715128_s_at,11715129_s_at,11715130_s_at,11715131_s_at,11715132_x_at,11715133_s_at,11715134_s_at,11715135_s_at,11715136_at,11715137_s_at,11715138_s_at,11715139_x_at,11715140_s_at,11715141_s_at,11715142_s_at,11715143_s_at,11715144_s_at,11715145_s_at,11715146_x_at,11715147_at,11715148_s_at,11715149_x_at,11715150_s_at,11715151_x_at,11715152_at,11715153_s_at,11715154_s_at,11715155_s_at,11715156_s_at,11715157_s_at,11715158_s_at,11715159_at,11715160_x_at,11715161_s_at,11715162_s_at,11715163_s_at,11715164_s_at,11715165_at,11715166_at,11715167_at,11715168_s_at,11715169_at,11715170_x_at,11715171_at,11715172_at,11715173_s_at,11715174_s_at,11715175_x_at,11715176_at,11715177_s_at,11715178_s_at,11715179_s_at,11715180_s_at,11715181_x_at,11715182_x_at,11715183_s_at,11715184_x_at,11715185_x_at,11715186_x_at,11715187_x_at,11715188_at,11715189_s_at,11715190_s_at,11715191_s_at,11715192_s_at,11715193_s_at,11715194_s_at,11715195_at,11715196_s_at,11715197_s_at,11715198_s_at,11715199_s_at,11715200_s_at,11715201_s_at,11715202_s_at,11715203_s_at,11715204_x_at,11715205_x_at,11715206_x_at,11715207_at,11715208_s_at,11715209_s_at,11715210_s_at,11715211_s_at,11715212_x_at,11715213_s_at,11715214_at,11715215_s_at,11715216_s_at,11715217_s_at,11715218_s_at,11715219_s_at,11715220_s_at,11715221_x_at,11715222_at,11715223_s_at,11715224_s_at,11715225_s_at,11715226_s_at,11715227_s_at,11715228_s_at,11715229_s_at,11715230_s_at,11715231_x_at,11715232_x_at,11715233_s_at,11715234_s_at,11715235_x_at,11715236_s_at,11715237_s_at,11715238_x_at,11715239_x_at,11715240_at,11715241_s_at,11715242_s_at,11715243_x_at,11715244_x_at,11715245_s_at,11715246_x_at,11715247_s_at,11715248_s_at,11715249_s_at,11715250_s_at,11715251_at,11715252_at,11715253_s_at,11715254_s_at,11715255_s_at,11715256_at,11715257_x_at,11715258_s_at,11715259_s_at,11715260_at,11715261_s_at,11715262_at,11715263_at,11715264_s_at,11715265_at,11715266_s_at,11715267_s_at,11715268_s_at,11715269_x_at,11715270_s_at,11715271_s_at,11715272_s_at,11715273_at,11715274_x_at,11715275_s_at,11715276_s_at,11715277_s_at,11715278_s_at,11715279_x_at,11715280_s_at,11715281_at,11715282_s_at,11715283_at,11715284_x_at,11715285_at,11715286_x_at,11715287_at,11715288_s_at,11715289_s_at,11715290_s_at,11715291_s_at,11715292_at,11715293_s_at,11715294_s_at,11715295_s_at,11715296_s_at,11715297_at,11715298_x_at,11715299_s_at,11715300_s_at,11715301_s_at,11715302_s_at,11715303_s_at,11715304_s_at,11715305_s_at,11715306_s_at,11715307_s_at,11715308_s_at,11715309_s_at,11715310_s_at,11715311_s_at,11715312_at,11715313_s_at,11715314_at,11715315_s_at,11715316_x_at,11715317_s_at,11715318_s_at,11715319_s_at,11715320_s_at,11715321_x_at,11715322_s_at,11715323_s_at,11715324_s_at,11715325_at,11715326_s_at,11715327_x_at,11715328_at,11715329_at,11715330_x_at,11715331_x_at,11715332_at,11715333_s_at,11715334_s_at,11715335_at,11715336_x_at,11715337_s_at,11715338_s_at,11715339_at,11715340_s_at,11715341_at,11715342_at,11715343_at,11715344_at,11715345_s_at,11715346_at,11715347_s_at,11715348_x_at,11715349_a_at,...,11764236_s_at,11764237_at,11764238_at,11764239_s_at,11764240_s_at,11764241_at,11764242_s_at,11764243_at,11764244_s_at,11764245_s_at,11764246_s_at,11764247_s_at,11764248_s_at,11764249_a_at,11764250_at,11764251_s_at,11764252_s_at,11764253_s_at,11764254_a_at,11764255_at,11764256_s_at,11764257_a_at,11764258_at,11764259_at,11764260_s_at,11764261_s_at,11764262_s_at,11764263_s_at,11764264_at,11764265_at,11764266_s_at,11764267_at,11764268_at,11764269_at,11764270_s_at,11764271_at,11764272_at,11764273_at,11764274_s_at,11764275_s_at,11764276_at,11764277_at,11764278_at,11764279_at,11764280_at,11764281_

матрица признаков экспрессии генов

In [288]:
X = ad_vs_cn_gene_exp

In [289]:
labels = ad_vs_cn['label']

In [290]:
X.columns


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at', '11715108_x_at', '11715109_at',
       ...
       'AFFX-r2-TagH_at', 'AFFX-r2-TagIN-3_at', 'AFFX-r2-TagIN-5_at', 'AFFX-r2-TagIN-M_at', 'AFFX-r2-TagJ-3_at', 'AFFX-r2-TagJ-5_at', 'AFFX-r2-TagO-3_at', 'AFFX-r2-TagO-5_at', 'AFFX-r2-TagQ-3_at', 'AFFX-r2-TagQ-5_at'], dtype='object', length=49386)

In [291]:
X.shape

(343, 49386)

In [292]:

labels.shape

(343,)

In [293]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV

In [294]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest


In [295]:
param_grid =[
    {'model__C': [0.01, 0.1,1],
     'model__class_weight': ['balanced',None],
     'model__kernel': ['linear'],
     'model__probability': [True],
     'model__random_state': [seed]}]
model_to_tune = Pipeline([    ("vt", VarianceThreshold()),
                             ( "scaler", StandardScaler()),
                          ("fs",SelectKBest(k=2000)),
                          ("pca", PCA(n_components =50)),
                            ("model", SVC(random_state=seed))])

Baseline model

In [296]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)


# Create a baseline logistic regression model
baseline_model = LogisticRegression()

X = ad_vs_cn_gene_exp  # Replace 'feature_columns' with the actual feature column names

# Outer cross-validation to compute the testing score
test_scores = cross_val_score(baseline_model, X, labels, cv=outer_cv, n_jobs=4, scoring='f1_macro')

print(f"The mean F1 macro score of the baseline model is: {np.mean(test_scores):.3f} ± {np.std(test_scores):.3f}")

The mean F1 macro score of the baseline model is: 0.609 ± 0.026


SVM model

In [297]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune, param_grid=param_grid, cv=inner_cv, n_jobs=4
)

X = ad_vs_cn_gene_exp

# Outer cross-validation to compute the testing score
test_score_lr = cross_val_score(model, X, labels, cv=outer_cv, n_jobs=4, scoring='f1_macro')
print(f"The mean score_svm using nested cross-validation is: "
      f"{test_score_lr.mean():.3f} ± {test_score_lr.std():.3f}")


The mean score_svm using nested cross-validation is: 0.575 ± 0.036


LogReg

In [304]:
param_grid =[
    {'model__C': [0.01, 0.1,1],
     'model__class_weight': ['balanced',None],
     'model__random_state': [seed]}]
model_to_tune_lr = Pipeline([    ("vt", VarianceThreshold()),
                             ( "scaler", StandardScaler()),
                          ("fs",SelectKBest(k=2000)),
                          ("pca", PCA(n_components =50)),
                            ("model", LogisticRegression ())])

In [305]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

model = GridSearchCV(
    estimator=model_to_tune_lr, param_grid=param_grid, cv=inner_cv, n_jobs=4
)

X = ad_vs_cn_gene_exp  # Replace 'feature_columns' with the actual feature column names

# Outer cross-validation to compute the testing score
test_scores = cross_val_score(model, X, labels, cv=outer_cv, n_jobs=4, scoring='f1_macro')

print(f"The mean F1 macro score of the baseline model is: {np.mean(test_scores):.3f} ± {np.std(test_scores):.3f}")

The mean F1 macro score of the baseline model is: 0.586 ± 0.027


In [298]:
# Fit the GridSearchCV to your data
model.fit(X, labels)

# Get the best model with the best hyperparameters
best_model = model.best_estimator_

# Access the hyperparameters of the best model
best_params = best_model.named_steps['model'].get_params()

# Remove the 'random_state' argument from the 'SVC' model constructor within the pipeline
reconstructed_model = Pipeline([
    ("vt", VarianceThreshold()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(k=2000)),
    ("pca", PCA(n_components=50)),
    ("model", SVC(**best_params))
])

# Fit the best model on the entire dataset
reconstructed_model.fit(X, labels)


# Extract feature importances if using a linear kernel (as mentioned in a previous response)
if best_params['kernel'] == 'linear':
    feature_names = X.columns  # Replace with the actual feature names if available
    feature_importances = reconstructed_model.named_steps['model'].coef_[0]

    # Sort the feature importances and get the indices of the top 10 features
    top_10_feature_indices = feature_importances.argsort()[-10:][::-1]

    # Get the names and importances of the top 10 features
    top_10_features = [(feature_names[i], feature_importances[i]) for i in top_10_feature_indices]

    print("Top 10 Most Important Features:")
    for feature, importance in top_10_features:
        print(f"{feature}: {importance:.4f}")

Top 10 Most Important Features:
11715126_s_at: 0.5055
11715110_at: 0.4527
11715136_at: 0.4152
11715107_s_at: 0.4147
11715147_at: 0.3738
11715119_s_at: 0.3566
11715140_s_at: 0.3486
11715132_x_at: 0.3464
11715145_s_at: 0.3303
11715104_s_at: 0.2514


In [299]:
# Assuming you have your best SVM model (best_model) as previously defined
kernel_used = best_model.named_steps['model'].kernel

print(f"The kernel used by the SVM model is: {kernel_used}")


The kernel used by the SVM model is: linear


### **ML for MCI vs CN**

In [ ]:
mci_vs_cn = genes[(genes['label'] == 2.0) | (genes['label'] == 1.0)]

In [ ]:
mci.shape

In [ ]:
cn.shape

In [ ]:
mci_vs_cn.shape

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest

# Определите модели для PCA, нормализации и логистической регрессии
pca = PCA()
scaler = StandardScaler()

In [ ]:
mci_vs_cn_gene_exp = mci_vs_cn.iloc[:, :-7]
mci_vs_cn_gene_exp.head()

In [ ]:
X_mci = mci_vs_cn_gene_exp.values

In [ ]:
labels_mci = mci_vs_cn['label']

In [ ]:
X_mci.shape

In [ ]:
labels_mci.shape

In [ ]:
param_grid_mci =[
    {'model__C': [0.01, 0.1,1],
     'model__class_weight': ['balanced',None],
     'model__kernel': ['linear'],
     'model__probability': [True],
     'model__random_state': [seed]}]
model_to_tune = Pipeline([    ("vt", VarianceThreshold()),
                             ( "scaler", StandardScaler()),
                          ("fs",SelectKBest(k=1000)),
                          ("pca", PCA(n_components =50)),
                            ("model", SVC(random_state=seed))])

Baseline model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Create a baseline logistic regression model
baseline_model = LogisticRegression()

X = X_mci  # Replace 'feature_columns' with the actual feature column names

# Outer cross-validation to compute the testing score
test_scores = cross_val_score(baseline_model, X, labels_mci, cv=outer_cv, n_jobs=4, scoring='f1_macro')

print(f"The mean F1 macro score of the baseline model is: {np.mean(test_scores):.3f} ± {np.std(test_scores):.3f}")

SVC model

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune, param_grid=param_grid_mci, cv=inner_cv, n_jobs=4
)

# Outer cross-validation to compute the testing score
test_score_lr = cross_val_score(model, X_mci, labels_mci, cv=outer_cv, n_jobs=4, scoring='f1_macro')
print(f"The mean score_svm using nested cross-validation is: "
      f"{test_score_lr.mean():.3f} ± {test_score_lr.std():.3f}")


### **MRI data preparation**

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest


In [ ]:
mri = pd.read_csv('UCSFFSX51_11_08_19_17Oct2023.csv')
mri.shape


In [ ]:
import pandas as pd

# Предположим, что у вас есть DataFrame df_mri и столбец 'EXAMDATE'
# Преобразуйте 'EXAMDATE' в строковый тип данных, если он не является таковым
mri['EXAMDATE'] = mri['EXAMDATE'].astype(str)

# Оставьте только первые 4 символа в столбце 'EXAMDATE'
mri['EXAMDATE'] = mri['EXAMDATE'].str[:4]


In [ ]:
mri.head()

In [ ]:
genes

In [ ]:
genes.shape

In [ ]:
import pandas as pd


#Выберите только строки в df_mri, у которых есть совпадение с 'rid' в df_genes
#mri_with_labels = mri[mri['RID'].isin(genes['RID'])]
mri_with_labels = pd.merge(mri, genes, left_on='RID', right_on='RID', how='left')
# Теперь объедините столбцы из df_genes с df_mri
mri_with_labels = pd.merge(mri, genes[['RID', 'subjectIDs', 'year', 'label', 'Phase', 'ID', 'VISDATE']], on='RID', how='inner')

mri_with_labels.shape


MRI data that has genetics

In [ ]:
mri_with_labels.head()

In [ ]:
unique_count = mri_with_labels['subjectIDs'].nunique()

if unique_count == len(mri_with_labels['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

In [ ]:

# Найдите дублирующиеся значения в столбце 'subjectIDs'
duplicates = mri_with_labels[mri_with_labels.duplicated(subset='subjectIDs', keep=False)]

# Выведите дублирующиеся значения
duplicates


Filter dublicates on years

In [ ]:
# Если тип данных столбца VISDATE не является строковым, преобразуйте его
#mri_with_labels['EXAMDATE'] = mri_with_labels['EXAMDATE'].astype(str)

# Теперь можно использовать .str accessor
#mri_with_labels['EXAMDATE'] = mri_with_labels['EXAMDATE'].str[:4]

# Преобразуйте столбцы year и VISDATE в строки, чтобы сравнивать их
mri_with_labels['year'] = mri_with_labels['year'].astype(str)
mri_with_labels['EXAMDATE'] = mri_with_labels['EXAMDATE'].astype(str)

# Создайте условие для сравнения year и VISDATE
condition = mri_with_labels['year'] == mri_with_labels['EXAMDATE']

# Удалите строки, где условие не выполняется
mri_with_labels = mri_with_labels[condition]

unique_count = mri_with_labels['subjectIDs'].nunique()

if unique_count == len(mri_with_labels['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

In [ ]:
mri_with_labels.shape

In [ ]:
subjects_no = mri_with_labels['label'].value_counts()
print(subjects_no)


In [ ]:
# Удаляем строки, где 'label' равно 5.0, 4.0 или 7.0
mri_with_labels = mri_with_labels[~mri_with_labels['label'].isin([5.0, 4.0, 7.0])]


In [ ]:
unique_count = mri_with_labels['subjectIDs'].nunique()

if unique_count == len(mri_with_labels['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

In [ ]:
has_nan = mri_with_labels.isna().any().any()
print(f"Есть ли NaN в таблице: {has_nan}")

In [ ]:
nan_rows = mri_with_labels[mri_with_labels.isna().any(axis=1)]

In [ ]:
columns_to_drop = ['LONISID', 'LONIUID', 'TEMPQC', 'FRONTQC', 'PARQC', 'INSULAQC', 'OCCQC', 'BGQC', 'CWMQC', 'VENTQC', 'LHIPQC', 'RHIPQC']
mri_with_labels = mri_with_labels.drop(columns=columns_to_drop)


In [ ]:
mri_with_labels.shape

In [ ]:
mri_with_labels_drop = mri_with_labels.dropna()
mri_with_labels_drop.shape

In [ ]:
unique_viscode2 = mri_with_labels['VISCODE2'].unique()
print(unique_viscode2)



In [ ]:
viscode2_counts = mri_with_labels['VISCODE2'].value_counts()
print(viscode2_counts)


In [ ]:
sc_mri_with_lables = mri_with_labels[mri_with_labels['VISCODE2'] == 'scmri']

In [ ]:
sc_mri_with_lables.shape

In [ ]:
unique_count = sc_mri_with_lables['subjectIDs'].nunique()

if unique_count == len(sc_mri_with_lables['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

### **Accelerated T1 data preparation**

In [ ]:
at1_sc_mri_with_labels = sc_mri_with_lables[sc_mri_with_lables['IMAGETYPE'] != 'Non-Accelerated T1']

In [ ]:
at1_sc_mri_with_labels.shape

In [ ]:
imgtype_counts = at1_sc_mri_with_labels['IMAGETYPE'].value_counts()
print(imgtype_counts)


In [ ]:
imgtype_counts = sc_mri_with_lables['IMAGETYPE'].value_counts()
print(imgtype_counts)


In [ ]:
unique_count = at1_sc_mri_with_labels['subjectIDs'].nunique()

if unique_count == len(at1_sc_mri_with_labels['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

In [ ]:

# Найдите дублирующиеся значения в столбце 'subjectIDs'
at1_sc_mri_with_labels_duplicates = at1_sc_mri_with_labels[at1_sc_mri_with_labels.duplicated(subset='subjectIDs', keep=False)]

# Выведите дублирующиеся значения
at1_sc_mri_with_labels_duplicates


In [ ]:
viscode2_counts = at1_sc_mri_with_labels['OVERALLQC'].value_counts()
print(viscode2_counts)


In [ ]:
at1_sc_mri_with_labels.shape

In [ ]:
import pandas as pd

# Проверяем, есть ли хотя бы один "Pass" в каждой группе "subjectIDs"
pass_in_group = at1_sc_mri_with_labels.groupby('subjectIDs')['OVERALLQC'].transform(lambda x: 'Pass' in x.values)

# Фильтруем строки, соответствующие вашим условиям
filtered_at1_sc_mri = at1_sc_mri_with_labels[~((at1_sc_mri_with_labels['OVERALLQC'].isin(['Partial', 'Hippocampus Only', 'Fail'])) & pass_in_group)]


filtered_at1_sc_mri - df в котором есть все данные, но у дупликатов quality = только Pass

In [ ]:
filtered_at1_sc_mri.shape

In [ ]:
columns_to_drop = ['COLPROT', 'RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'VERSION', 'IMAGEUID', 'IMAGETYPE', 'RUNDATE', 'STATUS', 'OVERALLQC', 'update_stamp', 'year', 'Phase', 'ID', 'VISDATE']
filtered_at1_sc_mri = filtered_at1_sc_mri.drop(columns=columns_to_drop)

In [ ]:
filtered_at1_sc_mri.head ()

In [ ]:
filtered_at1_sc_mri.shape

In [ ]:
# Найдите дублирующиеся значения в столбце 'subjectIDs'
filtered_at1_sc_mri_d = filtered_at1_sc_mri[filtered_at1_sc_mri.duplicated(subset='subjectIDs', keep=False)]

# Выведите дублирующиеся значения
filtered_at1_sc_mri_d


In [ ]:
unique_subject_ids_na = filtered_at1_sc_mri_d['subjectIDs'].unique()
print(unique_subject_ids_na)


In [ ]:
import pandas as pd

# Замените 'filtered_at1_sc_mri' на имя вашего DataFrame, если оно отличается
df = filtered_at1_sc_mri

# Получите уникальные значения из столбца "subjectIDs"
unique_subject_ids = df['subjectIDs'].unique()

# Пройдитесь по уникальным значениям и обновите DataFrame
for subject_id in unique_subject_ids_na:
    target_rows = df[df['subjectIDs'] == subject_id]
    if not target_rows.empty:
        # Найдите среднее значение по всем столбцам, кроме 'label'
        mean_values = target_rows.drop(columns=['label']).mean(numeric_only=True)

        # Обновите значения в первой строке (index=0)
        for col in mean_values.index:
            df.loc[df['subjectIDs'] == subject_id, col] = mean_values[col]

# Удалите дубликаты строк, оставив только первую строку
df = df.drop_duplicates(subset='subjectIDs', keep='first')
df.reset_index(drop=True, inplace=True)


In [ ]:
filtered_at1_sc_mri = df

In [ ]:
filtered_at1_sc_mri.shape

In [ ]:
filtered_at1_sc_mri.head()

In [ ]:
unique_count = filtered_at1_sc_mri['subjectIDs'].nunique()

if unique_count == len(filtered_at1_sc_mri['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

In [ ]:
# Найдите дублирующиеся значения в столбце 'subjectIDs'
filtered_at1_sc_mri_d = filtered_at1_sc_mri[filtered_at1_sc_mri.duplicated(subset='subjectIDs', keep=False)]

# Выведите дублирующиеся значения
filtered_at1_sc_mri_d


In [ ]:
filtered_at1_sc_mri.head()

### **MRI AD vs CN accelerated-T1**

In [ ]:
ad_only = filtered_at1_sc_mri[(filtered_at1_sc_mri['label'] == 3.0)]
ad_only.shape

In [ ]:
mci_only = filtered_at1_sc_mri[(filtered_at1_sc_mri['label'] == 2.0)]
mci_only.shape

In [ ]:
cn_only = filtered_at1_sc_mri[(filtered_at1_sc_mri['label'] == 1.0)]
cn_only.shape

In [ ]:
ad_vs_cn_mri = filtered_at1_sc_mri[(filtered_at1_sc_mri['label'] == 3.0) | (filtered_at1_sc_mri['label'] == 1.0)]

In [ ]:
columns_with_nan_ad_t1 = ad_vs_cn_mri.columns[ad_vs_cn_mri.isna().any()].tolist()
columns_with_nan_ad_t1

In [ ]:
columns_to_drop = ['ST8SV']
ad_vs_cn_mri = ad_vs_cn_mri.drop(columns=columns_to_drop)

In [ ]:
ad_vs_cn_mri_values = ad_vs_cn_mri.iloc[:, :-2]

In [ ]:
ad_vs_cn_mri_values.shape

In [ ]:
ad_vs_cn_mri_values.head()

Creating feature matrix

In [ ]:
X_ad_mri = ad_vs_cn_mri_values.values

In [ ]:
X_ad_mri.shape

In [ ]:
labels_ad_mri = ad_vs_cn_mri['label']

In [ ]:
labels_ad_mri.shape

Choosing params and preparing for OuterCV

In [ ]:
param_grid_ad_mri =[
    {'model__C': [0.01, 0.1,1],
     'model__class_weight': ['balanced',None],
     'model__kernel': ['linear', 'rbf'],
     'model__probability': [True],
     'model__random_state': [seed]}]
model_to_tune_ad_mri = Pipeline([    ("vt", VarianceThreshold()),
                             ( "scaler", StandardScaler()),
                          ("fs",SelectKBest(k='all')),
                          ("pca", PCA(n_components =21)),
                            ("model", SVC(random_state=seed))])

Baseline model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Create a baseline logistic regression model
baseline_model = LogisticRegression()

# Outer cross-validation to compute the testing score
test_scores = cross_val_score(baseline_model, X_ad_mri, labels_ad_mri, cv=outer_cv, n_jobs=4, scoring='f1_macro')

print(f"The mean F1 macro score of the baseline model is: {np.mean(test_scores):.3f} ± {np.std(test_scores):.3f}")

SVM model

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune_ad_mri, param_grid=param_grid_ad_mri, cv=inner_cv, n_jobs=4
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, X_ad_mri, labels_ad_mri, cv=outer_cv, n_jobs=4, scoring='f1_macro')
print(f"The mean score_svm using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")


### **MRI MCI vs CN accelerated-T1**

In [ ]:
mci_vs_cn_mri = filtered_at1_sc_mri[(filtered_at1_sc_mri['label'] == 2.0) | (filtered_at1_sc_mri['label'] == 1.0)]

In [ ]:
cn_only = filtered_at1_sc_mri[(filtered_at1_sc_mri['label'] == 1.0)]

In [ ]:
cn_only.shape

In [ ]:
mci_only = filtered_at1_sc_mri[(filtered_at1_sc_mri['label'] == 2.0)]

In [ ]:
mci_only.shape

In [ ]:
columns_with_nan_mci_t1 = mci_vs_cn_mri.columns[mci_vs_cn_mri.isna().any()].tolist()
columns_with_nan_mci_t1

In [ ]:
columns_to_drop = ['ST8SV']
mci_vs_cn_mri = mci_vs_cn_mri.drop(columns=columns_to_drop)

In [ ]:
mci_vs_cn_mri_values = mci_vs_cn_mri.iloc[:, :-2]

In [ ]:
mci_vs_cn_mri_values.shape

In [ ]:
ad_vs_cn_mri_values.head()

Creating feature matrix

In [ ]:
X_mci_mri = mci_vs_cn_mri_values.values

In [ ]:
X_mci_mri.shape

In [ ]:
labels_mci_mri = mci_vs_cn_mri['label']

In [ ]:
labels_mci_mri.shape

Choosing params and preparing for OuterCV

In [ ]:
param_grid_mci_mri =[
    {'model__C': [0.01, 0.1,1],
     'model__class_weight': ['balanced',None],
     'model__kernel': ['linear'],
     'model__probability': [True],
     'model__random_state': [seed]}]
model_to_tune_mci_mri = Pipeline([    ("vt", VarianceThreshold()),
                             ( "scaler", StandardScaler()),
                          ("fs",SelectKBest(k='all')),
                          ("pca", PCA(n_components =50)),
                            ("model", SVC(random_state=seed))])

Baseline model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Create a baseline logistic regression model
baseline_model = LogisticRegression()

# Outer cross-validation to compute the testing score
test_scores = cross_val_score(baseline_model, X_mci_mri, labels_mci_mri, cv=outer_cv, n_jobs=4, scoring='f1_macro')

print(f"The mean F1 macro score of the baseline model is: {np.mean(test_scores):.3f} ± {np.std(test_scores):.3f}")

SVM model

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune_mci_mri, param_grid=param_grid_mci_mri, cv=inner_cv, n_jobs=4
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, X_mci_mri, labels_mci_mri, cv=outer_cv, n_jobs=4, scoring='f1_macro')
print(f"The mean score_svm using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")


### **Non-accelerated data preparation**

In [ ]:
nat1_sc_mri_with_labels = sc_mri_with_lables[sc_mri_with_lables['IMAGETYPE'] != 'Accelerated T1']

In [ ]:
nat1_sc_mri_with_labels.shape

In [ ]:
imgtype_counts = nat1_sc_mri_with_labels['IMAGETYPE'].value_counts()
print(imgtype_counts)


In [ ]:
imgtype_counts = sc_mri_with_lables['IMAGETYPE'].value_counts()
print(imgtype_counts)


In [ ]:
unique_count = nat1_sc_mri_with_labels['subjectIDs'].nunique()

if unique_count == len(nat1_sc_mri_with_labels['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

In [ ]:
#Найдите дублирующиеся значения в столбце 'subjectIDs'
nat1_sc_mri_with_labels_duplicates = nat1_sc_mri_with_labels[nat1_sc_mri_with_labels.duplicated(subset='subjectIDs', keep=False)]

# Выведите дублирующиеся значения
nat1_sc_mri_with_labels_duplicates


In [ ]:
viscode2_counts = nat1_sc_mri_with_labels['OVERALLQC'].value_counts()
print(viscode2_counts)


In [ ]:
nat1_sc_mri_with_labels.shape

In [ ]:
import pandas as pd

# Проверяем, есть ли хотя бы один "Pass" в каждой группе "subjectIDs"
pass_in_group = nat1_sc_mri_with_labels.groupby('subjectIDs')['OVERALLQC'].transform(lambda x: 'Pass' in x.values)

# Фильтруем строки, соответствующие вашим условиям
filtered_nat1_sc_mri = nat1_sc_mri_with_labels[~((nat1_sc_mri_with_labels['OVERALLQC'].isin(['Partial', 'Hippocampus Only', 'Fail'])) & pass_in_group)]


filtered_at1_sc_mri - df в котором есть все данные, но у дупликатов quality = только Pass

In [ ]:
filtered_nat1_sc_mri.shape

In [ ]:
columns_to_drop = ['COLPROT', 'RID', 'VISCODE', 'VISCODE2', 'EXAMDATE', 'VERSION', 'IMAGEUID', 'IMAGETYPE', 'RUNDATE', 'STATUS', 'OVERALLQC', 'update_stamp', 'year', 'Phase', 'ID', 'VISDATE']
filtered_nat1_sc_mri = filtered_nat1_sc_mri.drop(columns=columns_to_drop)

In [ ]:
filtered_nat1_sc_mri.head ()

In [ ]:
# Найдите дублирующиеся значения в столбце 'subjectIDs'
filtered_nat1_sc_mri_d = filtered_nat1_sc_mri[filtered_nat1_sc_mri.duplicated(subset='subjectIDs', keep=False)]

# Выведите дублирующиеся значения
filtered_nat1_sc_mri_d


In [ ]:
unique_subject_ids = filtered_nat1_sc_mri_d['subjectIDs'].unique()
print(unique_subject_ids)


In [ ]:
import pandas as pd

# Замените 'filtered_at1_sc_mri' на имя вашего DataFrame, если оно отличается
df = filtered_nat1_sc_mri

# Получите уникальные значения из столбца "subjectIDs"
unique_subject_ids = df['subjectIDs'].unique()

# Пройдитесь по уникальным значениям и обновите DataFrame
for subject_id in unique_subject_ids:
    target_rows = df[df['subjectIDs'] == subject_id]
    if not target_rows.empty:
        # Найдите среднее значение по всем столбцам, кроме 'label'
        mean_values = target_rows.drop(columns=['label']).mean(numeric_only=True)

        # Обновите значения в первой строке (index=0)
        for col in mean_values.index:
            df.loc[df['subjectIDs'] == subject_id, col] = mean_values[col]

# Удалите дубликаты строк, оставив только первую строку
df = df.drop_duplicates(subset='subjectIDs', keep='first')
df.reset_index(drop=True, inplace=True)


In [ ]:
filtered_nat1_sc_mri = df

In [ ]:
filtered_nat1_sc_mri.shape

In [ ]:
filtered_nat1_sc_mri.head()

In [ ]:
unique_count = filtered_nat1_sc_mri['subjectIDs'].nunique()

if unique_count == len(filtered_nat1_sc_mri['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

In [ ]:
# Найдите дублирующиеся значения в столбце 'subjectIDs'
filtered_nat1_sc_mri_d = filtered_nat1_sc_mri[filtered_nat1_sc_mri.duplicated(subset='subjectIDs', keep=False)]

# Выведите дублирующиеся значения
filtered_nat1_sc_mri_d


In [ ]:
filtered_nat1_sc_mri.head()

In [ ]:
filtered_nat1_sc_mri.shape

In [ ]:
unique_count = filtered_nat1_sc_mri['subjectIDs'].nunique()

if unique_count == len(filtered_nat1_sc_mri['subjectIDs']):
    print("Значения в столбце 'subjectIDs' уникальны.")
else:
    print("Значения в столбце 'subjectIDs' не уникальны.")

In [ ]:
filtered_nat1_sc_mri.shape

In [ ]:
filtered_nat1_sc_mri = df

In [ ]:
filtered_nat1_sc_mri.head()

### **ML for non-accelerated ad vs cn**

In [ ]:
ad_only_na = filtered_nat1_sc_mri[(filtered_nat1_sc_mri['label'] == 3.0)]
ad_only_na.shape

In [ ]:
mci_only_na = filtered_nat1_sc_mri[(filtered_nat1_sc_mri['label'] == 2.0)]
mci_only_na.shape

In [ ]:
cn_only_na = filtered_nat1_sc_mri[(filtered_nat1_sc_mri['label'] == 1.0)]
cn_only_na.shape

In [ ]:
ad_vs_cn_mri_na = filtered_nat1_sc_mri[(filtered_nat1_sc_mri['label'] == 3.0) | (filtered_nat1_sc_mri['label'] == 1.0)]

In [ ]:
ad_vs_cn_mri_na.head()

In [ ]:
columns_with_nan_ad_na = ad_vs_cn_mri_na.columns[ad_vs_cn_mri_na.isna().any()].tolist()
columns_with_nan_ad_na

In [ ]:
columns_to_drop = ['ST125SV', 'ST28SA', 'ST87SA', 'ST8SV', 'ST147SV', 'ST148SV', 'ST149SV', 'ST150SV', 'ST151SV', 'ST152SV', 'ST153SV', 'ST154SV', 'ST155SV']
ad_vs_cn_mri_na = ad_vs_cn_mri_na.drop(columns=columns_to_drop)

In [ ]:
ad_vs_cn_mri_na_values = ad_vs_cn_mri_na.iloc[:, :-2]

In [ ]:
ad_vs_cn_mri_na_values.shape

In [ ]:
ad_vs_cn_mri_na_values.head()

Creating feature matrix

In [ ]:
X_ad_mri_na = ad_vs_cn_mri_na_values.values

In [ ]:
X_ad_mri_na.shape

In [ ]:
labels_ad_mri_na = ad_vs_cn_mri_na['label']

In [ ]:
labels_ad_mri_na.shape

In [ ]:
X_ad_mri_na

Choosing params and preparing for OuterCV

In [260]:
param_grid_ad_mri =[
    {'model__C': [0.01, 0.1,1],
     'model__class_weight': ['balanced',None],
     'model__kernel': ['linear'],
     'model__probability': [True],
     'model__random_state': [seed]}]
model_to_tune_ad_mri_na = Pipeline([    ("vt", VarianceThreshold()),
                             ( "scaler", StandardScaler()),
                          ("fs",SelectKBest(k='all')),
                          ("pca", PCA(n_components =21)),
                            ("model", SVC(random_state=seed))])

Baseline model

In [258]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Create a baseline logistic regression model
baseline_model = LogisticRegression()

# Outer cross-validation to compute the testing score
test_scores = cross_val_score(baseline_model, X_ad_mri_na, labels_ad_mri_na, cv=outer_cv, n_jobs=4, scoring='f1_macro')

print(f"The mean F1 macro score of the baseline model is: {np.mean(test_scores):.3f} ± {np.std(test_scores):.3f}")

The mean F1 macro score of the baseline model is: 0.876 ± 0.025


SVM model

In [261]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune_ad_mri_na, param_grid=param_grid_ad_mri, cv=inner_cv, n_jobs=4
)

X_ad_mri_na = ad_vs_cn_mri_na_values

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, X_ad_mri_na, labels_ad_mri_na, cv=outer_cv, n_jobs=4, scoring='f1_macro')
print(f"The mean score_svm using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score_svm using nested cross-validation is: 0.846 ± 0.006


In [ ]:
# Fit the GridSearchCV to your data
model.fit(X_ad_mri_na, labels_ad_mri_na)

# Get the best model with the best hyperparameters
best_model = model.best_estimator_

# Access the hyperparameters of the best model
best_params = best_model.named_steps['model'].get_params()

# Remove the 'random_state' argument from the 'SVC' model constructor within the pipeline
reconstructed_model = Pipeline([
    ("vt", VarianceThreshold()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(k='all')),
    ("pca", PCA(n_components=21)),
    ("model", SVC(**best_params))])

# Fit the best model on the entire dataset
reconstructed_model.fit(X_ad_mri_na, labels_ad_mri_na)


# Extract feature importances if using a linear kernel (as mentioned in a previous response)
if best_params['kernel'] == 'linear':
    feature_names = X_ad_mri_na.columns  # Replace with the actual feature names if available
    feature_importances = reconstructed_model.named_steps['model'].coef_[0]

    # Sort the feature importances and get the indices of the top 10 features
    top_10_feature_indices = feature_importances.argsort()[-10:][::-1]

    # Get the names and importances of the top 10 features
    top_10_features = [(feature_names[i], feature_importances[i]) for i in top_10_feature_indices]

    print("Top 10 Most Important Features:")
    for feature, importance in top_10_features:
        print(f"{feature}: {importance:.4f}")

# Note: If you are using an RBF kernel, you won't have feature importances to extract.


### **ML for non-accelerated mci vs cn**

In [ ]:
ad_only_na = filtered_nat1_sc_mri[(filtered_nat1_sc_mri['label'] == 3.0)]
ad_only_na.shape

In [ ]:
mci_only_na = filtered_nat1_sc_mri[(filtered_nat1_sc_mri['label'] == 2.0)]
mci_only_na.shape

In [ ]:
cn_only_na = filtered_nat1_sc_mri[(filtered_nat1_sc_mri['label'] == 1.0)]
cn_only_na.shape

In [ ]:
mci_vs_cn_mri_na = filtered_nat1_sc_mri[(filtered_nat1_sc_mri['label'] == 2.0) | (filtered_nat1_sc_mri['label'] == 1.0)]

In [ ]:
mci_vs_cn_mri_na.head()

In [ ]:
columns_with_nan_mci_na = mci_vs_cn_mri_na.columns[mci_vs_cn_mri_na.isna().any()].tolist()
columns_with_nan_mci_na

In [ ]:
columns_to_drop = ['ST28SA', 'ST87SA', 'ST8SV', 'ST147SV', 'ST148SV', 'ST149SV', 'ST150SV', 'ST151SV', 'ST152SV', 'ST153SV', 'ST154SV', 'ST155SV']
mci_vs_cn_mri_na = mci_vs_cn_mri_na.drop(columns=columns_to_drop)

In [ ]:
mci_vs_cn_mri_na

In [ ]:
mci_vs_cn_mri_na_values = mci_vs_cn_mri_na.iloc[:, :-2]

In [ ]:
mci_vs_cn_mri_na_values.shape

In [ ]:
mci_vs_cn_mri_na_values.head()

Creating feature matrix

In [ ]:
X_mci_mri_na = mci_vs_cn_mri_na_values.values

In [ ]:
X_mci_mri_na.shape

In [ ]:
labels_mci_mri_na = mci_vs_cn_mri_na['label']

In [ ]:
labels_mci_mri_na.shape

Choosing params and preparing for OuterCV

In [ ]:
param_grid_ad_mri =[{
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf'],
    'svc__class_weight': ['balanced', None],
    'svc__probability': [True],
    'svc__random_state': [14],
    'svc__gamma': ['scale', 'auto', 0.1, 1.0]  # Добавление gamma для ядра RBF
}]
model_to_tune_mci_mri_na = Pipeline([    ("vt", VarianceThreshold()),
                             ( "scaler", StandardScaler()),
                          ("fs",SelectKBest(k='all')),
                          ("pca", PCA(n_components =50)),
                            ("svc", SVC(random_state=seed))])

Baseline model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Create a baseline logistic regression model
baseline_model = LogisticRegression()

# Outer cross-validation to compute the testing score
test_scores = cross_val_score(baseline_model, X_mci_mri_na, labels_mci_mri_na, cv=outer_cv, n_jobs=4, scoring='f1_macro')

print(f"The mean F1 macro score of the baseline model is: {np.mean(test_scores):.3f} ± {np.std(test_scores):.3f}")

SVM model

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV

# Declare the inner and outer cross-validation strategies
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=model_to_tune_mci_mri_na, param_grid=param_grid_ad_mri, cv=inner_cv, n_jobs=4
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, X_mci_mri_na, labels_mci_mri_na, cv=outer_cv, n_jobs=4, scoring='f1_macro')
print(f"The mean score_svm using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")


### **Data preparation for MRI + GENES FUSION**

ad_vs_cn - таблица генов, нужно убрать столбцы ad_vs_cn_gene_exp = ad_vs_cn.iloc[:, :-7]

In [ ]:
ad_vs_cn.head()

In [ ]:
ad_vs_cn_mri_na.iloc[:, :-2]

In [ ]:
ad_vs_cn_mri_na

In [ ]:
merged_table_gene_mri_ad = pd.merge(ad_vs_cn_mri_na, ad_vs_cn, on='subjectIDs', how='inner')

In [ ]:
merged_table_gene_mri_ad.head()

In [ ]:
# Удалите указанные столбцы из таблицы
columns_to_drop = ["year", "label_y", "Phase", "ID", "RID", "VISDATE", "subjectIDs"]
merged_table = merged_table_gene_mri_ad.drop(columns=columns_to_drop)

# Результат будет сохранен в merged_table без указанных столбцов


In [ ]:
merged_table

In [ ]:
columns_drop = ["label_x"]

In [ ]:
data = merged_table.drop(columns=columns_drop)

In [ ]:
y = merged_table["label_x"]

### **MRI + GENES FUSION for AD**

In [248]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


In [251]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Создайте первый пайплайн
model_to_tune_ad_mri_na = Pipeline([
    ("vt", VarianceThreshold()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(k='2000')),
    ("pca", PCA(n_components=21))
])

# Создайте второй пайплайн
model_to_tune = Pipeline([
    ("vt", VarianceThreshold()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(k='all')),
    ("pca", PCA(n_components=50))
])

# Объедините их с использованием FeatureUnion
fusion_features = FeatureUnion([
    ('model_to_tune_ad_mri_na', model_to_tune_ad_mri_na),
    ('model_to_tune', model_to_tune)
])

# Теперь у вас есть объединенный пайплайн fusion_features, который включает оба пайплайна параллельно.


SVM model

In [252]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

# Create your two pipelines
model_to_tune_ad_mri_na = Pipeline([
    ("vt", VarianceThreshold()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(k='all')),
    ("pca", PCA(n_components=21))
])

model_to_tune = Pipeline([
    ("vt", VarianceThreshold()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(k=2000)),
    ("pca", PCA(n_components=50))
])

# Combine them using FeatureUnion
fusion_features = FeatureUnion([
    ('model_to_tune_ad_mri_na', model_to_tune_ad_mri_na),
    ('model_to_tune', model_to_tune)
])

# Inner and outer cross-validation
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Define your param_grid for hyperparameter tuning
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf'],
    'svc__class_weight': ['balanced', None],
    'svc__probability': [True],
    'svc__random_state': [14],
    'svc__gamma': ['scale', 'auto', 0.1, 1.0]  # Добавление gamma для ядра RBF
}


# Create a pipeline with the fusion_features and the SVM classifier
model = Pipeline([
    ('fusion_features', fusion_features),
    ('svc', SVC())
])

# Use your original data and labels for cross_val_score
test_score_svm = cross_val_score(model, data, y, cv=outer_cv, n_jobs=4, scoring='accuracy')
print(f"The mean score_svm using nested cross-validation is: {test_score_svm.mean():.3f} ± {test_score_svm.std():.3f}")


The mean score_svm using nested cross-validation is: 0.756 ± 0.033


Baseline model

In [253]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler



# Create a simple SVM model with default settings
baseline_model = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic', LogisticRegression ())
])

# Inner and outer cross-validation
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Use your original data and labels for cross_val_score
test_score_baseline = cross_val_score(baseline_model, data, y, cv=outer_cv, n_jobs=4, scoring='accuracy')

print(f"The mean baseline accuracy using nested cross-validation is: {test_score_baseline.mean():.3f} ± {test_score_baseline.std():.3f}")


The mean baseline accuracy using nested cross-validation is: 0.744 ± 0.024


### **Data preparation for MRI + GENES FUSION**

ad_vs_cn - таблица генов, нужно убрать столбцы ad_vs_cn_gene_exp = ad_vs_cn.iloc[:, :-7]

In [262]:
mci_vs_cn.head()

,11715100_at,11715101_s_at,11715102_x_at,11715103_x_at,11715104_s_at,11715105_at,11715106_x_at,11715107_s_at,11715108_x_at,11715109_at,11715110_at,11715111_s_at,11715112_at,11715113_x_at,11715114_x_at,11715115_s_at,11715116_s_at,11715117_x_at,11715118_s_at,11715119_s_at,11715120_s_at,11715121_s_at,11715122_at,11715123_s_at,11715124_s_at,11715125_at,11715126_s_at,11715127_s_at,11715128_s_at,11715129_s_at,11715130_s_at,11715131_s_at,11715132_x_at,11715133_s_at,11715134_s_at,11715135_s_at,11715136_at,11715137_s_at,11715138_s_at,11715139_x_at,11715140_s_at,11715141_s_at,11715142_s_at,11715143_s_at,11715144_s_at,11715145_s_at,11715146_x_at,11715147_at,11715148_s_at,11715149_x_at,11715150_s_at,11715151_x_at,11715152_at,11715153_s_at,11715154_s_at,11715155_s_at,11715156_s_at,11715157_s_at,11715158_s_at,11715159_at,11715160_x_at,11715161_s_at,11715162_s_at,11715163_s_at,11715164_s_at,11715165_at,11715166_at,11715167_at,11715168_s_at,11715169_at,11715170_x_at,11715171_at,11715172_at,11715173_s_at,11715174_s_at,11715175_x_at,11715176_at,11715177_s_at,11715178_s_at,11715179_s_at,11715180_s_at,11715181_x_at,11715182_x_at,11715183_s_at,11715184_x_at,11715185_x_at,11715186_x_at,11715187_x_at,11715188_at,11715189_s_at,11715190_s_at,11715191_s_at,11715192_s_at,11715193_s_at,11715194_s_at,11715195_at,11715196_s_at,11715197_s_at,11715198_s_at,11715199_s_at,11715200_s_at,11715201_s_at,11715202_s_at,11715203_s_at,11715204_x_at,11715205_x_at,11715206_x_at,11715207_at,11715208_s_at,11715209_s_at,11715210_s_at,11715211_s_at,11715212_x_at,11715213_s_at,11715214_at,11715215_s_at,11715216_s_at,11715217_s_at,11715218_s_at,11715219_s_at,11715220_s_at,11715221_x_at,11715222_at,11715223_s_at,11715224_s_at,11715225_s_at,11715226_s_at,11715227_s_at,11715228_s_at,11715229_s_at,11715230_s_at,11715231_x_at,11715232_x_at,11715233_s_at,11715234_s_at,11715235_x_at,11715236_s_at,11715237_s_at,11715238_x_at,11715239_x_at,11715240_at,11715241_s_at,11715242_s_at,11715243_x_at,11715244_x_at,11715245_s_at,11715246_x_at,11715247_s_at,11715248_s_at,11715249_s_at,11715250_s_at,11715251_at,11715252_at,11715253_s_at,11715254_s_at,11715255_s_at,11715256_at,11715257_x_at,11715258_s_at,11715259_s_at,11715260_at,11715261_s_at,11715262_at,11715263_at,11715264_s_at,11715265_at,11715266_s_at,11715267_s_at,11715268_s_at,11715269_x_at,11715270_s_at,11715271_s_at,11715272_s_at,11715273_at,11715274_x_at,11715275_s_at,11715276_s_at,11715277_s_at,11715278_s_at,11715279_x_at,11715280_s_at,11715281_at,11715282_s_at,11715283_at,11715284_x_at,11715285_at,11715286_x_at,11715287_at,11715288_s_at,11715289_s_at,11715290_s_at,11715291_s_at,11715292_at,11715293_s_at,11715294_s_at,11715295_s_at,11715296_s_at,11715297_at,11715298_x_at,11715299_s_at,11715300_s_at,11715301_s_at,11715302_s_at,11715303_s_at,11715304_s_at,11715305_s_at,11715306_s_at,11715307_s_at,11715308_s_at,11715309_s_at,11715310_s_at,11715311_s_at,11715312_at,11715313_s_at,11715314_at,11715315_s_at,11715316_x_at,11715317_s_at,11715318_s_at,11715319_s_at,11715320_s_at,11715321_x_at,11715322_s_at,11715323_s_at,11715324_s_at,11715325_at,11715326_s_at,11715327_x_at,11715328_at,11715329_at,11715330_x_at,11715331_x_at,11715332_at,11715333_s_at,11715334_s_at,11715335_at,11715336_x_at,11715337_s_at,11715338_s_at,11715339_at,11715340_s_at,11715341_at,11715342_at,11715343_at,11715344_at,11715345_s_at,11715346_at,11715347_s_at,11715348_x_at,11715349_a_at,...,11764243_at,11764244_s_at,11764245_s_at,11764246_s_at,11764247_s_at,11764248_s_at,11764249_a_at,11764250_at,11764251_s_at,11764252_s_at,11764253_s_at,11764254_a_at,11764255_at,11764256_s_at,11764257_a_at,11764258_at,11764259_at,11764260_s_at,11764261_s_at,11764262_s_at,11764263_s_at,11764264_at,11764265_at,11764266_s_at,11764267_at,11764268_at,11764269_at,11764270_s_at,11764271_at,11764272_at,11764273_at,11764274_s_at,11764275_s_at,11764276_at,11764277_at,11764278_at,11764279_at,11764280_at,11764281_at,11764282_x_at,11764283_s_at,11764284_at,11764285_at,11764286_x_at,11764287_at,11764288_at

In [263]:
mci_vs_cn_mri_na.iloc[:, :-2]

,ST101SV,ST102CV,ST102SA,ST102TA,ST102TS,ST103CV,ST103SA,ST103TA,ST103TS,ST104CV,ST104SA,ST104TA,ST104TS,ST105CV,ST105SA,ST105TA,ST105TS,ST106CV,ST106SA,ST106TA,ST106TS,ST107CV,ST107SA,ST107TA,ST107TS,ST108CV,ST108SA,ST108TA,ST108TS,ST109CV,ST109SA,ST109TA,ST109TS,ST10CV,ST110CV,ST110SA,ST110TA,ST110TS,ST111CV,ST111SA,ST111TA,ST111TS,ST112SV,ST113CV,ST113SA,ST113TA,ST113TS,ST114CV,ST114SA,ST114TA,ST114TS,ST115CV,ST115SA,ST115TA,ST115TS,ST116CV,ST116SA,ST116TA,ST116TS,ST117CV,ST117SA,ST117TA,ST117TS,ST118CV,ST118SA,ST118TA,ST118TS,ST119CV,ST119SA,ST119TA,ST119TS,ST11SV,ST120SV,ST121CV,ST121SA,ST121TA,ST121TS,ST124SV,ST125SV,ST127SV,ST128SV,ST129CV,ST129SA,ST129TA,ST129TS,ST12SV,ST130CV,ST130SA,ST130TA,ST130TS,ST13CV,ST13SA,ST13TA,ST13TS,ST14CV,ST14SA,ST14TA,ST14TS,ST15CV,ST15SA,ST15TA,ST15TS,ST16SV,ST17SV,ST18SV,ST1SV,ST21SV,ST23CV,ST23SA,ST23TA,ST23TS,ST24CV,ST24SA,ST24TA,ST24TS,ST25CV,ST25SA,ST25TA,ST25TS,ST26CV,ST26SA,ST26TA,ST26TS,ST29SV,ST2SV,ST30SV,ST31CV,ST31SA,ST31TA,ST31TS,ST32CV,ST32SA,ST32TA,ST32TS,ST34CV,ST34SA,ST34TA,ST34TS,ST35CV,ST35SA,ST35TA,ST35TS,ST36CV,ST36SA,ST36TA,ST36TS,ST37SV,ST38CV,ST38SA,ST38TA,ST38TS,ST39CV,ST39SA,ST39TA,ST39TS,ST3SV,ST40CV,ST40SA,ST40TA,ST40TS,ST42SV,ST43CV,ST43SA,ST43TA,ST43TS,ST44CV,ST44SA,ST44TA,ST44TS,ST45CV,ST45SA,ST45TA,ST45TS,ST46CV,ST46SA,ST46TA,ST46TS,ST47CV,ST47SA,ST47TA,ST47TS,ST48CV,ST48SA,ST48TA,ST48TS,ST49CV,ST49SA,ST49TA,ST49TS,ST4SV,ST50CV,ST50SA,ST50TA,ST50TS,ST51CV,ST51SA,ST51TA,ST51TS,ST52CV,ST52SA,ST52TA,ST52TS,ST53SV,ST54CV,ST54SA,ST54TA,ST54TS,ST55CV,ST55SA,ST55TA,ST55TS,ST56CV,ST56SA,ST56TA,ST56TS,ST57CV,ST57SA,ST57TA,ST57TS,ST58CV,ST58SA,ST58TA,ST58TS,ST59CV,ST59SA,ST59TA,ST59TS,ST5SV,ST60CV,ST60SA,ST60TA,ST60TS,ST61SV,ST62CV,ST62SA,ST62TA,ST62TS,ST65SV,ST66SV,ST68SV,ST69SV,ST6SV,ST70SV,ST71SV,ST72CV,ST72SA,ST72TA,ST72TS,ST73CV,ST73SA,ST73TA,ST73TS,ST74CV,ST74SA,ST74TA,ST74TS,ST75SV,ST76SV,ST77SV,ST7SV,ST80SV,ST82CV,ST82SA,ST82TA,ST82TS,ST83CV,ST83SA,ST83TA,ST83TS,ST84CV,ST84SA,ST84TA,ST84TS,ST85CV,ST85SA,ST85TA,ST85TS,ST88SV,ST89SV,ST90CV,ST90SA,ST90TA,ST90TS,ST91CV,ST91SA,ST91TA,ST91TS,ST93CV,ST93SA,ST93TA,ST93TS,ST94CV,ST94SA,ST94TA,ST94TS,ST95CV,ST95SA,ST95TA,ST95TS,ST96SV,ST97CV,ST97SA,ST97TA,ST97TS,ST98CV,ST98SA,ST98TA,ST98TS,ST99CV,ST99SA,ST99TA,ST99TS,ST9SV,ST131HS,ST132HS,ST133HS,ST134HS,ST135HS,ST136HS,ST137HS,ST138HS,ST139HS,ST140HS,ST141HS,ST142HS,ST143HS,ST144HS,ST145HS,ST146HS
0,1652.0,3835.0,1449.0,2.394,0.629,1785.0,628.0,2.495,0.612,3732.0,1348.0,2.539,0.490,2388.0,700.0,2.672,0.778,3914.0,1402.0,2.351,0.602,2265.0,1646.0,1.482,0.443,7318.0,3373.0,1.974,0.579,3400.0,1317.0,2.446,0.685,1488060.0,12471.0,4649.0,2.482,0.564,9167.0,3620.0,2.368,0.630,4732.0,2254.0,766.0,2.524,0.604,14035.0,5659.0,2.121,0.636,16536.0,5594.0,2.576,0.605,13069.0,4921.0,2.316,0.594,10331.0,3253.0,2.769,0.621,8736.0,3150.0,2.585,0.620,1704.0,335.0,3.512,0.897,525.0,6469,1039.0,381.0,2.447,0.422,3828,77.0,972.0,1618.0,6338.0,2093.0,3.014,0.877,1416.0,7121.0,2549.0,2.814,0.930,2802.0,1080.0,2.627,0.387,2079.0,698.0,2.865,0.596,7220.0,2703.0,2.471,0.503,3087.0,49354.0,15512.0,23000.0,1350.0,3138.0,1505.0,1.933,0.429,1763.0,363.0,3.493,0.570,464.0,118.0,2.793,0.411,10007.0,2913.0,2.904,0.707,3919,1100.0,183.0,11728.0,4099.0,2.621,0.572,10689.0,3120.0,3.028,0.659,2601.0,934.0,2.724,0.806,10669.0,4158.0,2.379,0.580,6073.0,2282.0,2.551,0.651,6025.0,6889.0,3422.0,1.874,0.594,4577.0,1683.0,2.568,0.761,569.0,10815.0,3149.0,2.916,0.585,1874.0,2940.0,1191.0,2.303,0.536,1790.0,634.0,2.369,0.756,3581.0,1347.0,2.516,0.460,1708.0,525.0,2.757,0.539,3504.0,1232.0,2.449,0.483,2009.0,1400.0,1.595,0.439,8330,3633.0,1.999,0.654,589.0,3631.0,1274.0,2.713,0.561,12766.0,4601.0,2.556,0.563,8851.0,3434.0,2.414,0.640,4966.0,2473.0,791.0,2.823,0.519,14362.0,5447.0,2.360,0.542,18829.0,6257.0,2.669,0.539,12473.0,4895.0,2.247,0.587,12461.0,3832.0,2.821,0.718,10644.0,3771.0,2.596,0.573,603.0,2092.0,444.0,3.485,0.703,6446.0,1312.0,521.0,2.221,0.499,3757.0,90.0,104.0,276.0,1175.0,613.0,1437.0,2163.0,860.

In [ ]:
mci_vs_cn_mri_na

In [264]:
merged_table_gene_mri_mci = pd.merge(mci_vs_cn_mri_na, mci_vs_cn, on='subjectIDs', how='inner')

In [265]:
merged_table_gene_mri_mci.head()

,ST101SV,ST102CV,ST102SA,ST102TA,ST102TS,ST103CV,ST103SA,ST103TA,ST103TS,ST104CV,ST104SA,ST104TA,ST104TS,ST105CV,ST105SA,ST105TA,ST105TS,ST106CV,ST106SA,ST106TA,ST106TS,ST107CV,ST107SA,ST107TA,ST107TS,ST108CV,ST108SA,ST108TA,ST108TS,ST109CV,ST109SA,ST109TA,ST109TS,ST10CV,ST110CV,ST110SA,ST110TA,ST110TS,ST111CV,ST111SA,ST111TA,ST111TS,ST112SV,ST113CV,ST113SA,ST113TA,ST113TS,ST114CV,ST114SA,ST114TA,ST114TS,ST115CV,ST115SA,ST115TA,ST115TS,ST116CV,ST116SA,ST116TA,ST116TS,ST117CV,ST117SA,ST117TA,ST117TS,ST118CV,ST118SA,ST118TA,ST118TS,ST119CV,ST119SA,ST119TA,ST119TS,ST11SV,ST120SV,ST121CV,ST121SA,ST121TA,ST121TS,ST124SV,ST125SV,ST127SV,ST128SV,ST129CV,ST129SA,ST129TA,ST129TS,ST12SV,ST130CV,ST130SA,ST130TA,ST130TS,ST13CV,ST13SA,ST13TA,ST13TS,ST14CV,ST14SA,ST14TA,ST14TS,ST15CV,ST15SA,ST15TA,ST15TS,ST16SV,ST17SV,ST18SV,ST1SV,ST21SV,ST23CV,ST23SA,ST23TA,ST23TS,ST24CV,ST24SA,ST24TA,ST24TS,ST25CV,ST25SA,ST25TA,ST25TS,ST26CV,ST26SA,ST26TA,ST26TS,ST29SV,ST2SV,ST30SV,ST31CV,ST31SA,ST31TA,ST31TS,ST32CV,ST32SA,ST32TA,ST32TS,ST34CV,ST34SA,ST34TA,ST34TS,ST35CV,ST35SA,ST35TA,ST35TS,ST36CV,ST36SA,ST36TA,ST36TS,ST37SV,ST38CV,ST38SA,ST38TA,ST38TS,ST39CV,ST39SA,ST39TA,ST39TS,ST3SV,ST40CV,ST40SA,ST40TA,ST40TS,ST42SV,ST43CV,ST43SA,ST43TA,ST43TS,ST44CV,ST44SA,ST44TA,ST44TS,ST45CV,ST45SA,ST45TA,ST45TS,ST46CV,ST46SA,ST46TA,ST46TS,ST47CV,ST47SA,ST47TA,ST47TS,ST48CV,ST48SA,ST48TA,ST48TS,ST49CV,ST49SA,ST49TA,ST49TS,ST4SV,ST50CV,ST50SA,ST50TA,ST50TS,ST51CV,ST51SA,ST51TA,ST51TS,ST52CV,ST52SA,ST52TA,ST52TS,ST53SV,ST54CV,ST54SA,ST54TA,ST54TS,ST55CV,ST55SA,ST55TA,ST55TS,ST56CV,ST56SA,ST56TA,ST56TS,ST57CV,ST57SA,ST57TA,ST57TS,ST58CV,ST58SA,ST58TA,ST58TS,ST59CV,ST59SA,ST59TA,ST59TS,ST5SV,ST60CV,ST60SA,ST60TA,ST60TS,ST61SV,ST62CV,ST62SA,ST62TA,ST62TS,ST65SV,ST66SV,ST68SV,ST69SV,ST6SV,ST70SV,ST71SV,ST72CV,ST72SA,ST72TA,ST72TS,ST73CV,ST73SA,...,11764242_s_at,11764243_at,11764244_s_at,11764245_s_at,11764246_s_at,11764247_s_at,11764248_s_at,11764249_a_at,11764250_at,11764251_s_at,11764252_s_at,11764253_s_at,11764254_a_at,11764255_at,11764256_s_at,11764257_a_at,11764258_at,11764259_at,11764260_s_at,11764261_s_at,11764262_s_at,11764263_s_at,11764264_at,11764265_at,11764266_s_at,11764267_at,11764268_at,11764269_at,11764270_s_at,11764271_at,11764272_at,11764273_at,11764274_s_at,11764275_s_at,11764276_at,11764277_at,11764278_at,11764279_at,11764280_at,11764281_at,11764282_x_at,11764283_s_at,11764284_at,11764285_at,11764286_x_at,11764287_at,11764288_at,11764289_x_at,11764290_at,11764291_a_at,11764292_x_at,200000_PM_s_at,200001_PM_at,200002_PM_at,200003_PM_s_at,200004_PM_at,200005_PM_at,200006_PM_at,200007_PM_at,200008_PM_s_at,200009_PM_at,200010_PM_at,200011_PM_s_at,200012_PM_x_at,200013_PM_at,200014_PM_s_at,200015_PM_s_at,200016_PM_x_at,200017_PM_at,200018_PM_at,200019_PM_s_at,200020_PM_at,200021_PM_at,200022_PM_at,200023_PM_s_at,200024_PM_at,200025_PM_s_at,200026_PM_at,200027_PM_at,200028_PM_s_at,200029_PM_at,200030_PM_s_at,200031_PM_s_at,200032_PM_s_at,200033_PM_at,200034_PM_s_at,200035_PM_at,200036_PM_s_at,200037_PM_s_at,200038_PM_s_at,200039_PM_s_at,200040_PM_at,200041_PM_s_at,200042_PM_at,200043_PM_at,200044_PM_at,200045_PM_at,200046_PM_at,200047_PM_s_at,200048_PM_s_at,200049_PM_at,200050_PM_at,200051_PM_at,200052_PM_s_at,200053_PM_at,200054_PM_at,200055_PM_at,200056_PM_s_at,200057_PM_s_at,200058_PM_s_at,200059_PM_s_at,200060_PM_s_at,200061_PM_s_at,200062_PM_s_at,200063_PM_s_at,200064_PM_at,200065_PM_s_at,200066_PM_at,200067_PM_x_at,200068_PM_s_at,200069_PM_at,200070_PM_at,200071_PM_at,200072_PM_s_at,200073_PM_s_at,200074_PM_s_at,200075_PM_s_at,200076_PM_s_at,200077_PM_s_at,200078_PM_s_at,200079_PM_s_at,200080_PM_s_at,200081_PM_s_at,200082_PM_s_at,200083_PM_at,200084_PM_at,200085_PM_s_at,200086_PM_s_at,200087_PM_s_at,200088_PM_x_at,200089_PM_s_at,200090_PM_at,200091_PM_s_at,200092_PM_s_at,200093_PM_s_at,200094_PM_s_at,200095_PM_x_at,200096_PM_s_at,200097_PM_s_at,200098_PM_s_at,200099_PM_s_at,AFFX-DapX-3_at,AFFX-DapX-5_at,AFFX-DapX-M_at,AFFX-HSAC07/X00351_3_at,AFFX-HSAC

In [267]:
# Удалите указанные столбцы из таблицы
columns_to_drop = ["year", "label_y", "Phase", "ID", "RID", "VISDATE", "subjectIDs"]
merged_table_mci = merged_table_gene_mri_mci.drop(columns=columns_to_drop)

# Результат будет сохранен в merged_table без указанных столбцов


In [ ]:
merged_table

In [268]:
columns_drop = ["label_x"]

In [269]:
data = merged_table_mci.drop(columns=columns_drop)

In [270]:
y = merged_table_mci["label_x"]

### **MRI + GENES FUSION for MCI**

In [271]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


SVM model

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold, SelectKBest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV

# Create your two pipelines
model_to_tune_ad_mri_na = Pipeline([
    ("vt", VarianceThreshold()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(k=2000)),
    ("pca", PCA(n_components=21))
])

model_to_tune = Pipeline([
    ("vt", VarianceThreshold()),
    ("scaler", StandardScaler()),
    ("fs", SelectKBest(k=2000)),
    ("pca", PCA(n_components=50))
])

# Combine them using FeatureUnion
fusion_features = FeatureUnion([
    ('model_to_tune_ad_mri_na', model_to_tune_ad_mri_na),
    ('model_to_tune', model_to_tune)
])

# Inner and outer cross-validation
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Define your param_grid for hyperparameter tuning
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf'],
    'svc__class_weight': ['balanced', None],
    'svc__probability': [True],
    'svc__random_state': [14],
    'svc__gamma': ['scale', 'auto', 0.1, 1.0]  # Добавление gamma для ядра RBF
}

# Create a pipeline with the fusion_features and the SVM classifier
model = Pipeline([
    ('fusion_features', fusion_features),
    ('svc', SVC())])

model = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=inner_cv, n_jobs=4
)




# Use your original data and labels for cross_val_score
test_score_svm = cross_val_score(model, data, y, cv=outer_cv, n_jobs=4, scoring='accuracy')
print(f"The mean score_svm using nested cross-validation is: {test_score_svm.mean():.3f} ± {test_score_svm.std():.3f}")


Baseline model

In [273]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler



# Create a simple SVM model with default settings
baseline_model = Pipeline([
    ('scaler', StandardScaler()),
    ('logistic', LogisticRegression ()),

])

# Inner and outer cross-validation
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Use your original data and labels for cross_val_score
test_score_baseline = cross_val_score(baseline_model, data, y, cv=outer_cv, n_jobs=4, scoring='accuracy')

print(f"The mean baseline accuracy using nested cross-validation is: {test_score_baseline.mean():.3f} ± {test_score_baseline.std():.3f}")


The mean baseline accuracy using nested cross-validation is: 0.646 ± 0.026


In [306]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler

# Inner and outer cross-validation
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1380)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=1380)

# Define your param_grid for hyperparameter tuning
param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__class_weight': ['balanced', None],
    'svc__random_state': [14]
}

# Create a pipeline with the fusion_features and the SVM classifier
model = Pipeline([
    ('fusion_features', fusion_features),
    ('svc', LogisticRegression())])

model = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=inner_cv, n_jobs=4
)

# Use your original data and labels for cross_val_score
test_score_baseline = cross_val_score(model, data, y, cv=outer_cv, n_jobs=4, scoring='accuracy')

print(f"The mean baseline accuracy using nested cross-validation is: {test_score_baseline.mean():.3f} ± {test_score_baseline.std():.3f}")




The mean baseline accuracy using nested cross-validation is: 0.654 ± 0.054
